In [34]:
import pandas as pd
import torch
import umap
from sklearn.cluster import KMeans

In [2]:
df = pd.read_csv('elasticsearch_data.csv')

In [3]:
df.head()

,url,title,published_date,content,tags,keywords,label,crawled_date
0,https://www.zoomit.ir/buying-guide/272487-best...,راهنمای خرید بهترین گوشی موبایل - تیر ۱۴۰۳ - ا...,1403-04-04 20:40:00,با‌‌ توجه‌‌‌‌ به افزایش بی‌سابقه و افسارگسیخته...,[],[],computer-learning,2024-06-24T20:56:02.038218
1,https://www.zoomit.ir/buying-guide/422495-best...,بهترین کولر آبی بازار ایران [خرداد ۱۴۰۳],1403-03-28 21:30:00,اگر در مناطقی با آب‌وهوای معتدل و نسبتاً خشک ز...,[],[],computer-learning,2024-06-24T20:56:04.426830
2,https://www.zoomit.ir/buying-guide/127190-best...,بهترین ساعت های هوشمند و مچ بند های هوشمند باز...,1403-03-27 16:45:00,تا چند سال قبل اگر قصد خرید \nساعت هوشمند\n یا...,[],[],computer-learning,2024-06-24T20:56:05.675861
3,https://www.zoomit.ir/computer-learning/420381...,چگونه کامپیوتر را از طریق گوشی خاموش کنیم؟ - ز...,1403-02-14 15:30:00,فرض کنید درحال‌ خروج از خانه یادتان می‌آید که ...,[],[],computer-learning,2024-06-24T20:55:58.824341
4,https://www.zoomit.ir/howto/347269-download-yo...,آموزش دانلود از یوتیوب با گوشی های اندرویدی و ...,1403-04-01 12:30:00,بعضی مواقع برای دسترسی آسان‌ و آفلاین به ویدئو...,[],[],computer-learning,2024-06-24T20:55:59.457658


In [16]:
labels = []
labels = df['label'].tolist()

In [15]:
df['label'] = df['label'].replace('technology', 'mobile')

In [17]:
print(set(labels))

{'mobile', 'laptop', 'computer-learning'}


In [29]:
# !pip install sentence_transformers
!pip uninstall umap
!pip install --upgrade umap-learn

In [9]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

all_embeddings = embedding_model.encode(df['content'].tolist(), convert_to_tensor=True)

if not isinstance(all_embeddings, torch.Tensor):
    all_embeddings_tensor = torch.stack(all_embeddings)
else:
    all_embeddings_tensor = all_embeddings
print(all_embeddings_tensor.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


torch.Size([75, 384])


In [32]:
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=50).fit(all_embeddings_tensor)
umap_embeddings = umap_model.fit_transform(all_embeddings_tensor.cpu().numpy())

AttributeError: module 'umap' has no attribute 'UMAP'

In [38]:
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans_labels = kmeans.fit_predict(all_embeddings_tensor)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [39]:
kmeans_cluster_sentences = {}
for label, sentence in zip(kmeans_labels, df['content']):
    if label not in kmeans_cluster_sentences:
        kmeans_cluster_sentences[label] = []
    kmeans_cluster_sentences[label].append(sentence)

In [40]:
import csv

csv_file_name = 'kmeans_clusters.csv'

with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Cluster', 'Tweet'])

    for cluster, sentences in kmeans_cluster_sentences.items():
        for sentence in sentences:
            writer.writerow([cluster, sentence])